In [5]:
import numpy as np
import matplotlib.pyplot as plt
from tueplots import figsizes
from tueplots import axes

#plitting specifications
plt.rcParams.update(axes.lines())
plt.rcParams.update({"figure.dpi": 600})
figsize_config = figsizes.aaai2024_half(nrows=2, ncols=1)
plt.rcParams.update(figsize_config)
color_cycler = plt.rcParams['axes.prop_cycle']
colors = color_cycler.by_key()['color']

In [6]:
# Create a simple symmetric positive definite (p.d.) 3x3 matrix
A = np.array([[2., -1., 0.],
              [-1., 2., -1.],
              [0., -1., 2.]])

# Check that it is symmetric
print("Symmetric:", np.allclose(A, A.T))

# Check positive definiteness by trying Cholesky decomposition (raises if not p.d.)
try:
    np.linalg.cholesky(A)
    print("A is positive definite.")
except np.linalg.LinAlgError:
    print("A is not positive definite.")

# Alternatively, check that all eigenvalues are positive
eigvals = np.linalg.eigvalsh(A)
print("Eigenvalues:", eigvals)
print("All eigenvalues positive:", np.all(eigvals > 0))


Symmetric: True
A is positive definite.
Eigenvalues: [0.58578644 2.         3.41421356]
All eigenvalues positive: True


In [7]:
L = np.linalg.cholesky(A)
print("Cholesky decomposition L:\n", L)
print("Reconstructed A from L @ L.T:\n", L @ L.T)


Cholesky decomposition L:
 [[ 1.41421356  0.          0.        ]
 [-0.70710678  1.22474487  0.        ]
 [ 0.         -0.81649658  1.15470054]]
Reconstructed A from L @ L.T:
 [[ 2. -1.  0.]
 [-1.  2. -1.]
 [ 0. -1.  2.]]
